Převody formátu - rasterizace a vektorizace
----------------------------------------------------------------

**Převod vektoru na rastr:**

In [ ]:
library(raster)
dam<-raster("damaged.tif")

In [ ]:
dam

In [ ]:
dam[dam==0]<-NA

In [ ]:
plot(dam)

Rastrová data lze převést na polygony pomocí `rasterToPolygons`. Funkce je ale pomalá, je třeba zohlednit velikost rastru a případné použití alternativy (viz https://johnbaumgartner.wordpress.com/2012/07/26/getting-rasters-into-shape-from-r/) 

In [ ]:
poly <- rasterToPolygons(dam,dissolve = T)

In [ ]:
poly

In [ ]:
library(rgdal)
writeOGR(poly,"dam.shp",layer = "dam.shp", driver = 'ESRI Shapefile')

**Převod rastru na vektor:**

In [ ]:
library(rgdal)
chru<-readOGR("../../data/protected_areas-etrs.shp")

In [ ]:
chru

In [ ]:
plot(chru)

In [ ]:
library(rgeos)
?gUnion

Nejdříve vyčstíme polygon sjednocením všech polygonů do jednoho. Sousední polygony vytvoří jeden polygon, a všechny polygony dohromady vytvoří jeden multi-feature polygon.

In [ ]:
chru<-gUnion(chru, chru)

In [ ]:
chru_b<-gBuffer(chru,width=0)

In [ ]:
chru

In [ ]:
plot(chru)

In [ ]:
writeOGR(chru,"chru.shp",layer = "chru.shp", driver = 'ESRI Shapefile')

Nejdříve je nutné vytvořit prázdný rastr

In [ ]:
chru@proj4string
#nebo
#crs(chru)

In [ ]:
#rozliseni
r_res<-1000
#souradnicovy system, prevzaty z puvodni vrstvy
r_crs<-crs(chru)
#rozsah (extent), prevzaty z puvodni vrstvy
r_ext<-extent(chru)

In [ ]:
empty_r<-raster(resolution=r_res,ext=r_ext,crs=r_crs)

In [ ]:
empty_r

In [ ]:
r_chru <- rasterize(chru, empty_r)

In [ ]:
r_chru

In [ ]:
plot(r_chru)

In [ ]:
writeRaster(r_chru,"r_chru.tif")